In [1]:
import boto3
import json
import time

# 입력받은 버킷의 파일 조회
def bucket_element(name):
    my_bucket = boto3.resource('s3').Bucket(name)
    result = []
    
    for element in my_bucket.objects.all():
        result.append(element.key)
    
    return result

# 로컬에 있는 파일을 rekognition celebrity 분석
def recognition_celebrities(photo):
    client = boto3.client('rekognition')
    name = ''
    
    with open(photo, 'rb') as image:
        response = client.recognize_celebrities(Image={'Bytes': image.read()})
   
    for celebrity in response['CelebrityFaces']:
        name = celebrity['Name']
    
    return name 
    
# 프로젝트 - 영상내에 존재하는 유명인사 찾기
def recognition_celebrities_video(bucket, key):
    client=boto3.client('rekognition')
    response = client.start_celebrity_recognition(
        Video={'S3Object': {'Bucket': bucket, 
                            'Name': key}})
    startJobId=response['JobId']
    
    print('Start JobId: ' + startJobId)
    print()
    
    return startJobId
    
# 위 함수를 실행하고 반복적으로 실행. 넘겨받은 ID를 가지고 계속 get하다보면 어느 순간
# 'IN_PROGRESS'가 'SUCCEEDED'로 바뀜
def recognition_celebrities_video_get(Id):
    client = boto3.client('rekognition')
    response = client.get_celebrity_recognition(JobId=Id)
    
    return response

# 영상 분석 결과에서 유명인사 리스트를 가져온다.
def listing_celebrity(response, list_celebrity):
    for celebrityRecognition in response['Celebrities']:
        cel_name = str(celebrityRecognition['Celebrity']['Name'])
        if not cel_name in list_celebrity:
            list_celebrity.append(cel_name)

# 평점이 가장 높은 작품을 사용자에게 추천함
def maximum_index(grade_list):
    m = 0
    index = 0
    result = 0
    
    for grade in grade_list:
        if m < grade:
            m = grade
            result = index
        index = index + 1
    
    return result

In [2]:
# S3 bucket에 있는 동영상을 분석하여 결과를 저장

bucket_name = 'netflix-video-storage' #aws s3 버켓 이름
bucket_files = [] # 버켓 내 있는 영상 파일명
list_celebrity = [] # 버켓 내 있는 모든 영상 분석 결과

bucket_files = bucket_element(bucket_name)
length = len(bucket_files)
for i in bucket_files:
    list_celebrity.append(list())
    
i = 0
for file in bucket_files:
    print("filename : ", file)
    Id = recognition_celebrities_video(bucket_name, file)
    response = recognition_celebrities_video_get(Id)
        
    print("Analyzing Video...(Id : {})".format(Id))
    while response['JobStatus'] != 'SUCCEEDED':
        time.sleep(10)
        response = recognition_celebrities_video_get(Id)
        
    print("Analysis complete!!!(Id : {})".format(Id))
    print("Save Celebrities...")
    listing_celebrity(response, list_celebrity[i])
    i = i + 1
    print()

filename :  1박2일-103회.mp4
Start JobId: 66aff4559cb2ccd61405fe4ec54031888932c0de14927845fdec857530ce8789

Analyzing Video...(Id : 66aff4559cb2ccd61405fe4ec54031888932c0de14927845fdec857530ce8789)
Analysis complete!!!(Id : 66aff4559cb2ccd61405fe4ec54031888932c0de14927845fdec857530ce8789)
Save Celebrities...

filename :  1박2일-151회.mp4
Start JobId: 41ff5069b2d374f30b64dfd3f4122b5b4d4f4479003203f05ef9c7d76ab48501

Analyzing Video...(Id : 41ff5069b2d374f30b64dfd3f4122b5b4d4f4479003203f05ef9c7d76ab48501)
Analysis complete!!!(Id : 41ff5069b2d374f30b64dfd3f4122b5b4d4f4479003203f05ef9c7d76ab48501)
Save Celebrities...

filename :  1박2일-30회.mp4
Start JobId: f1f3967a4eb369106431cee9333749b56229bac02cff7eb3e221deade209ecc3

Analyzing Video...(Id : f1f3967a4eb369106431cee9333749b56229bac02cff7eb3e221deade209ecc3)
Analysis complete!!!(Id : f1f3967a4eb369106431cee9333749b56229bac02cff7eb3e221deade209ecc3)
Save Celebrities...

filename :  FSET-426.mp4
Start JobId: 7c51dc8de57f87635bddfab0e49c81eb0745c7f

In [3]:
# main함수
def main(image):
    user_input = image #내가 찾고 싶은 유명인사
    celebrity = ''
    index_list = [] # 유명인사가 출연한 작품의 index리스트 -> bucket_files의 리스트
    celebrity_pro = [] # 유명인사가 출연한 작품을 저장할 리스트
    grade_list = [] # 출연작들의 평점을 저장할 리스트
    
    celebrity = recognition_celebrities(user_input)
    print("user input result = ", celebrity)
    
    i = 0
    for celebrities in list_celebrity:
        if celebrity in celebrities:
            index_list.append(i)
        i = i + 1
        
    for index in index_list:
        file = bucket_files[index]
        file = (file.split('.'))[0]
        celebrity_pro.append(file)
    
    
    f = open('data.txt', 'r', encoding='utf-8')
    i = 0
    while True:
        line = f.readline().strip('\n')
        if not line: 
            break
        
        if i == len(celebrity_pro):
            break
        
        if line != celebrity_pro[i]:
            line = f.readline().strip('\n')
            continue
        
        grade = f.readline().strip('\n')
        grade_list.append(float(grade))
        i = i + 1
                      
    f.close()
    
    i = 0
    index = maximum_index(grade_list)
    print("Appeared Program List of {}".format(celebrity))
    for pro in celebrity_pro:
        print("{}. {}".format(i+1, pro))
        i = i + 1
        
    print()
    print("recommendation : {}".format(celebrity_pro[index]))

In [5]:
main('Robert Downey Jr.jpg')

user input result =  Robert Downey Jr.
Appeared Program List of Robert Downey Jr.
1. Sherlock Holmes
2. The Story of Doctor Dolittle

recommendation : Sherlock Holmes


In [6]:
main('Donnie Yen.jpg')

user input result =  Donnie Yen
Appeared Program List of Donnie Yen
1. 엽문1
2. 엽문3

recommendation : 엽문1


In [7]:
main('Dwayne Johnson.jpg')

user input result =  Dwayne Johnson
Appeared Program List of Dwayne Johnson
1. 분노의 질주_ 홉스&쇼

recommendation : 분노의 질주_ 홉스&쇼


In [8]:
main('Jason Statham.jpg')

user input result =  Jason Statham
Appeared Program List of Jason Statham
1. Transporter 2
2. 분노의 질주_ 홉스&쇼

recommendation : 분노의 질주_ 홉스&쇼


In [9]:
# 런닝맨 20회가 잡힌 이유 -> 영상 중간에 X맨 영상이 나오고
# 거기에 강호동의 얼굴이 나오기 때문으로 보인다...
main('Kang Ho-dong.png')

user input result =  Kang Ho-dong
Appeared Program List of Kang Ho-dong
1. 1박2일-103회
2. 1박2일-151회
3. 1박2일-30회
4. 런닝맨 20회

recommendation : 1박2일-151회


In [10]:
main('Lee Seung-gi.png')

user input result =  Lee Seung-gi
Appeared Program List of Lee Seung-gi
1. 1박2일-103회
2. 1박2일-151회
3. 1박2일-30회

recommendation : 1박2일-151회


In [11]:
main('Lee Soo-geun.jpg')

user input result =  Lee Soo-geun
Appeared Program List of Lee Soo-geun
1. 1박2일-103회
2. 1박2일-151회
3. 1박2일-30회

recommendation : 1박2일-151회


In [12]:
main('Ma Dong-Seok.jpg')

user input result =  Ma Dong-Seok
Appeared Program List of Ma Dong-Seok
1. 범죄도시
2. 원더퓰 고스트
3. 챔피온

recommendation : 범죄도시


In [13]:
main('Yoo Jae Suk.jpg')

user input result =  Yoo Jae Suk
Appeared Program List of Yoo Jae Suk
1. 런닝맨 20회
2. 런닝맨 512화
3. 해피투게더 613화

recommendation : 런닝맨 512화


In [14]:
# AV배우는 유명인사로 잡히지 않는다.
# 당연히 영상 분석의 결과에도 잡히지 않는다.
# AV배우의 얼굴을 인식하는 인식기를 따로 구현할 필요가 있다...
main('uehara ai.jpg')

user input result =  Rina Suzuki
Appeared Program List of Rina Suzuki



IndexError: list index out of range